In [ ]:
import sys
import torch
sys.path.append("../")
from irradiance.utilities.temperature_response import TemperatureResponse

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
temp_res = TemperatureResponse(temp_resp_path='/home/andres_munoz_j/2024-HL-SPI3S-MEGS-AI/irradiance/data/aia_temp_resp.genx')

In [68]:
n_channels = len(temp_res.response)
temp_intervals = torch.linspace(temp_res.response[94]['log_T'][0], temp_res.response[94]['log_T'][-1], n_channels).to(device)
dT = torch.pow(10, temp_intervals[1:])-torch.pow(10, temp_intervals[0:-1])
temp_intervals.shape

7

In [69]:
dem_matrix = torch.zeros((n_channels, n_channels)).to(device)
dem_matrix.shape

torch.Size([7, 7])

In [76]:
for i, wavelength in enumerate(temp_res.response.keys()):
    # print(i)
    interpolator = temp_res.response[wavelength]['interpolator'](temp_intervals)
    print(interpolator)
    for j in range(n_channels):
        print(j, temp_intervals[j])
        if j==0:
            dem_matrix[i,j] = dT[0]*interpolator[0]/2
            print(interpolator[0])
        elif j==n_channels-1:
            dem_matrix[i,j] = dT[j-1]*interpolator[j]/2
            print(interpolator[j])
        else:
            dem_matrix[i,j] = (dT[j-1]+dT[j])*interpolator[j]/2
            print(interpolator[j])

tensor([1.2155e-36, 4.8358e-30, 1.1396e-27, 1.1031e-27, 8.0656e-29, 4.3767e-29,
        2.2748e-29], device='cuda:0')
0 tensor(4., device='cuda:0')
tensor(1.2155e-36, device='cuda:0')
1 tensor(4.8333, device='cuda:0')
tensor(4.8358e-30, device='cuda:0')
2 tensor(5.6667, device='cuda:0')
tensor(1.1396e-27, device='cuda:0')
3 tensor(6.5000, device='cuda:0')
tensor(1.1031e-27, device='cuda:0')
4 tensor(7.3333, device='cuda:0')
tensor(8.0656e-29, device='cuda:0')
5 tensor(8.1667, device='cuda:0')
tensor(4.3767e-29, device='cuda:0')
6 tensor(9., device='cuda:0')
tensor(2.2748e-29, device='cuda:0')
tensor([9.2337e-37, 6.9564e-30, 1.4062e-25, 2.1557e-27, 3.6115e-27, 4.6069e-28,
        2.3546e-28], device='cuda:0')
0 tensor(4., device='cuda:0')
tensor(9.2337e-37, device='cuda:0')
1 tensor(4.8333, device='cuda:0')
tensor(6.9564e-30, device='cuda:0')
2 tensor(5.6667, device='cuda:0')
tensor(1.4062e-25, device='cuda:0')
3 tensor(6.5000, device='cuda:0')
tensor(2.1557e-27, device='cuda:0')
4 tens

In [77]:
torch.linalg.inv(dem_matrix)

tensor([[ 4.0173e+28, -2.1582e+27,  5.2815e+26,  1.4035e+26, -3.2403e+27,
         -7.2344e+24,  5.1307e+27],
        [-3.7959e+20,  2.3080e+19, -5.7203e+18, -1.5319e+18,  2.7425e+19,
          5.8675e+18, -4.0665e+19],
        [-1.7791e+20,  9.4072e+18, -1.2262e+18, -6.1348e+17,  1.3253e+19,
          1.3888e+16, -2.0636e+19],
        [-5.4087e+20,  2.8798e+19, -7.1557e+18, -1.8926e+18,  4.3226e+19,
          3.2667e+16, -5.9331e+19],
        [-1.9242e+20,  1.3735e+19, -3.1539e+18, -5.7557e+17,  1.0782e+19,
          2.4031e+15, -1.0923e+19],
        [ 2.8475e+21, -2.1400e+20,  4.6605e+19,  1.3944e+19, -1.5249e+20,
         -6.9311e+14,  1.3319e+20],
        [-5.3846e+21,  4.2777e+20, -9.2152e+19, -2.8044e+19,  2.7584e+20,
         -4.2206e+16, -2.1254e+20]], device='cuda:0')